# Premier League Match Analysis: A Statistical Exploration

This project analyzes Premier League match results from the 2006-2007 season to the 2023-2024 season. It serves as a portfolio piece to demonstrate a wide array of foundational data analysis skills, from basic statistical concepts to more advanced techniques.

![Premier League Match Analysis](picture.jpg)

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns